In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_replace, split, size, element_at, lit,expr,trim,regexp_extract,udf
from pyspark.sql.types import IntegerType, StringType, ArrayType
import re

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Mi aplicación PySpark") \
    .getOrCreate()

df = spark.read.csv("C:/Users/adria/OneDrive/Python/My-Python/Basico/Basico-Adriana/onpiece1.csv", header=True, inferSchema=True)

#spark.stop()
# Visualiza los datos
print("Primeras 5 filas:")
df.show(5)

#Descripción de los datos
print ("Descripcion datos")
df.describe()

Primeras 5 filas:
+----------+---------------------+--------------------+--------------------+---------------+-------+------------+---------------+---------+------------+--------------------+----+----------------+----------+------------+-------+----+------------+--------------------+------------------+-------+---------+------------+-----------+---------+-------+----------+-------+---------+-----------+----------------+-------+----+---------+---------------+-------+----+------------------+----------+-----------+---------------+--------+------+-----------+-----------+-------------+----+------+--------+---------------+------+----------+
|Unnamed: 0|Official English Name|               Debut|        Affiliations|    Occupations| Status|    Birthday|     Unnamed: 9|Character|      Origin|           Residence| Age|          Height|Blood Type|English Name|Meaning|Type|Age at Death|LiveAction Portrayal|First Appearance19|Admiral|Main Ship|Total Bounty|Affiliation|Patriarch|Captain|Occupation|L

DataFrame[summary: string, Unnamed: 0: string, Official English Name: string, Debut: string, Affiliations: string, Occupations: string, Status: string, Birthday: string, Unnamed: 9: string, Character: string, Origin: string, Residence: string, Age: string, Height: string, Blood Type: string, English Name: string, Meaning: string, Type: string, Age at Death: string, LiveAction Portrayal: string, First Appearance19: string, Admiral: string, Main Ship: string, Total Bounty: string, Affiliation: string, Patriarch: string, Captain: string, Occupation: string, Leaders: string, Residency: string, 12th Leader: string, 13th Tobe Leader: string, Kingdom: string, Boss: string, Matriarch: string, GovernorGeneral: string,  Number: string, Size: string, First appearance37: string, Birth Name: string, Japanese VA: string, Funi English VA: string, Features: string, Weight: string, Fruit Debut: string, Usage Debut: string, Previous User: string,  key: string, Vessel: string, Homeland: string, Completio

In [77]:
spark = SparkSession.builder.appName("Contar filas").getOrCreate()
num_filas = df.count()
print(f"La tabla tiene {num_filas} filas.")

schema = df.schema  # Obtiene el esquema del DataFrame
for field in schema.fields:
    print(f"Columna: {field.name}, Tipo: {field.dataType}, Longitud máxima: {field.metadata.get('maxlength', 'N/A')}")

La tabla tiene 1394 filas.
Columna: Unnamed: 0, Tipo: IntegerType(), Longitud máxima: N/A
Columna: Official English Name, Tipo: StringType(), Longitud máxima: N/A
Columna: Debut, Tipo: StringType(), Longitud máxima: N/A
Columna: Affiliations, Tipo: StringType(), Longitud máxima: N/A
Columna: Occupations, Tipo: StringType(), Longitud máxima: N/A
Columna: Status, Tipo: StringType(), Longitud máxima: N/A
Columna: Birthday, Tipo: StringType(), Longitud máxima: N/A
Columna: Unnamed: 9, Tipo: StringType(), Longitud máxima: N/A
Columna: Character, Tipo: StringType(), Longitud máxima: N/A
Columna: Origin, Tipo: StringType(), Longitud máxima: N/A
Columna: Residence, Tipo: StringType(), Longitud máxima: N/A
Columna: Age, Tipo: StringType(), Longitud máxima: N/A
Columna: Height, Tipo: StringType(), Longitud máxima: N/A
Columna: Blood Type, Tipo: StringType(), Longitud máxima: N/A
Columna: English Name, Tipo: StringType(), Longitud máxima: N/A
Columna: Meaning, Tipo: StringType(), Longitud máxima:

In [80]:
df.createOrReplaceTempView("Onpiece1_vw")
#df_seleccionado = spark.sql("SELECT `Official English Name`, Age, `Total Bounty` FROM Onpiece1_VW")
df_seleccionado = spark.sql("SELECT DISTINCT `Official English Name`,`Total Bounty` FROM Onpiece1_VW WHERE `Total Bounty` is not null")
df_seleccionado.show()

+---------------------+--------------------+
|Official English Name|        Total Bounty|
+---------------------+--------------------+
|      Big Mom Pirates|Main Crew:At leas...|
|           Barto Club|At least 267,000,...|
| Animal Kingdom Pi...|At least 11,179,1...|
|  Happosui Army (Viz)|At least 752,000,...|
|        Rocks Pirates|At least 570,000,000|
|       Bonney Pirates|At least 320,000,...|
| Black Beard Pirat...|Main Crew:At leas...|
|        Heart Pirates|At least 3,000,00...|
|        Four Emperors|14,233,900,00015,...|
|          Kid Pirates|At least 3,200,00...|
| Buggy's Band of P...|At least 3,320,00...|
|    Yonta Maria Fleet|At least 148,000,000|
| Straw Hat Grand F...|At least 10,380,0...|
| Straw Hat Pirates...|8,816,001,0003,16...|
|    Beautiful Pirates|At least 397,000,...|
+---------------------+--------------------+



In [104]:
# Eliminar etiquetas [bounty X]
df = df_seleccionado.withColumn("Total Bounty_N", regexp_replace(col("Total Bounty"), r"\[bounty \d+\]", ""))

# Verificar la eliminación de etiquetas
#print("Después de eliminar etiquetas:")
#df.select("Official English Name", "Total Bounty","Total Bounty_N").show(200,truncate=False)

# Dividir en recompensas
df = df.withColumn("Bounties", split(col("Total Bounty_N"), r"At least|, "))

# Verificar la división en recompensas
#print("Después de dividir en recompensas:")
#df.select("Official English Name","Total Bounty_N","Bounties").show(200,truncate=False)



# Extraer la última recompensa
df = df.withColumn("Ultima Recompensa", element_at(col("Bounties"), -1))

# Verificar la extracción de la última recompensa
#print("Después de extraer la última recompensa:")
#df.select("Official English Name","Bounties" ,"Ultima Recompensa").show(200,truncate=False)

# Limpiar la última recompensa eliminando cualquier cosa que no sea un número
df = df.withColumn("Ultima Recompensa_N", regexp_replace(col("Ultima Recompensa"), "[^0-9]", ""))


# Verificar los valores limpios en la columna Ultima Recompensa antes de la conversión a entero
#print("Después de limpiar la última recompensa:")
#df.select("Official English Name", "Ultima Recompensa_N","Ultima Recompensa").show(200,truncate=False)
schema = df.schema  # Obtiene el esquema del DataFrame
for field in schema.fields:
    print(f"Columna: {field.name}, Tipo: {field.dataType}, Longitud máxima: {field.metadata.get('maxlength', 'N/A')}")

# Convertir a entero
df = df.withColumn("Ultima Recompensa",col("Ultima Recompensa_N").cast("bigint"))  # Convertir a entero
# df = df.withColumn(
#     "Ultima Recompensa",
#     when(
#         col("Ultima Recompensa").rlike("^[0-9]+$"),  # Verificar si solo contiene números
#         col("Ultima Recompensa").cast("int")  # Convertir a entero
#     ).otherwise(0)  # Si no es un número válido, asignar 0
# )

# Verificar la columna Ultima Recompensa después de la conversión
#print("Después de convertir a entero:")
#df.select("Official English Name","Ultima Recompensa_N", "Ultima Recompensa").show(200,truncate=False)

# Filtrar el DataFrame para valores mayores a 10,000 y menores o iguales a 30,000,000
df_filtered = df.filter(
    (col("Ultima Recompensa") > 10000) & (col("Ultima Recompensa") <= 11401100000)
)

# Mostrar el resultado
print("Después de filtrar los valores:")
df_filtered.select("Official English Name", "Ultima Recompensa").show(truncate=False)

Columna: Official English Name, Tipo: StringType(), Longitud máxima: N/A
Columna: Total Bounty, Tipo: StringType(), Longitud máxima: N/A
Columna: Total Bounty_N, Tipo: StringType(), Longitud máxima: N/A
Columna: Bounties, Tipo: ArrayType(StringType(), False), Longitud máxima: N/A
Columna: Ultima Recompensa, Tipo: StringType(), Longitud máxima: N/A
Columna: Ultima Recompensa_N, Tipo: StringType(), Longitud máxima: N/A
Después de filtrar los valores:
+-------------------------------------------------------------------------------------------------------------+-----------------+
|Official English Name                                                                                        |Ultima Recompensa|
+-------------------------------------------------------------------------------------------------------------+-----------------+
|Big Mom Pirates                                                                                              |11044500000      |
|Barto Club                